## Ejercicio - Regresión Viviendas Barcelona

- Utilizando el dataset de publicaciones de venta de viviendas en Barcelona _**viviendas_barcelona.parquet**_, entrena un modelo de regresión lineal múltiple que sea capaz de calcular el precio de venta:

    - Antes de crear el modelo y entrenarlo, haz un **EDA** de los datos.
        - ¿Qué columnas pueden ser las más importantes?
        - ¿Como se distribuyen los precios?
        - ¿Qué columnas sería factible limpiar?
        - ¿Qué significan los **np.nan**?
     
    - Cuando entiendas bien los datos con los que estás tratando, puedes proceder a realizar un preprocesamiento:
        - Elimina duplicados.
        - Elimina columnas innecesarias
        - Trata los valores perdidos **np.nan**.
        - Maneja los outliers como consideres.
        - Realiza transformaciones a los datos que consideres necesarias.
        - Codifica los datos categóricos para que el modelo los entienda (pásalos a números).
        - Considera normalizar los datos antes de utilizarlos para el entreno.
        
    - Entrena el modelo con un subconjunto de 70-80% del dataset.
      
    - Utiliza los datos restantes para comprobar cómo de bueno es el modelo:
        - Calcula **R2**, **MAE** y **MSE**.
        
- Importa las librerías que vayas a necesitar.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Modelo
from sklearn.linear_model import LinearRegression

# Herramientas de preprocesamiento
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, TargetEncoder, OneHotEncoder

# Split
from sklearn.model_selection import train_test_split

# Métricas
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

df = pd.read_parquet("../Data/viviendas_barcelona.parquet")

In [4]:
df.head()

,price,title,province,location,lat,lng,agency,updated,timestamp,id_1,...,Consumo,Habitaciones,Referencia,Antigüedad,Garaje,Terraza,Se aceptan mascotas,Ascensor,Tipo de casa,Superficie útil
0,52000.0,"Piso en venta en Avenida Casetes, nº 33",barcelona,"Avenida Casetes, nº 33. Centre Vila (Vilafranc...",41.348350,1.701970,Inmuebles de Aliseda Inmobiliaria,1.672874e+09,1.697761e+15,34241677924,...,A B C D E F G,3,ASR1-SRB0000036956,None,None,None,None,None,None,None
1,315000.0,Casa adosada en venta en Vilafranca del Penedes,barcelona,Castellet i la Gornal,41.276503,1.652229,Inmuebles de Engel & Völkers Barcelona,1.672701e+09,1.697761e+15,23357117200,...,None,4,IF76306-I-00VJM3-W-02OAMJ,None,1,Terraza,None,None,Adosada,None
2,650000.0,Casa unifamiliar en venta en Sant Sadurní d'Anoia,barcelona,Sant Sadurní d'Anoia,41.426682,1.792630,Inmuebles de SAFTI ESPAÑA,1.674861e+09,1.697761e+14,32584643160,...,None,5,SA3541-REF_11510,None,2,Terraza,None,None,Unifamiliar,220 m²
3,315000.0,Casa en venta en Carrer de Sant Pere Molanta,barcelona,Carrer de Sant Pere Molanta. Olèrdola,41.318658,1.754891,Inmuebles de ALARCON PROPIEDADES,1.673997e+09,1.697761e+15,19202422630,...,None,3,4017-4751,Entre 30 y 50 años,1,Terraza,Se aceptan mascotas,None,None,100 m²
4,209900.0,"Chalet en venta en Calle Carrer Serral Llarg, ...",barcelona,"Calle Carrer Serral Llarg, nº 107. Olesa de Bo...",41.369700,1.876770,Inmuebles de FINCAS MARINA,1.674256e+09,1.697761e+15,35859429926,...,None,3,SA3007-08353,None,None,Terraza,None,None,None,398 m²


In [8]:
df["agency"].nunique()

1329

,price,title,province,location,lat,lng,agency,updated,timestamp,id_1,...,Consumo,Habitaciones,Referencia,Antigüedad,Garaje,Terraza,Se aceptan mascotas,Ascensor,Tipo de casa,Superficie útil
138,NaN,None,barcelona,None,NaN,NaN,None,NaN,1.697762e+15,25092950882,...,None,None,None,None,None,None,None,None,None,None
2437,NaN,None,barcelona,None,NaN,NaN,None,NaN,1.697776e+15,32512388509,...,None,None,None,None,None,None,None,None,None,None
2746,NaN,None,barcelona,None,NaN,NaN,None,NaN,1.697778e+15,32519223008,...,None,None,None,None,None,None,None,None,None,None
3207,NaN,None,barcelona,None,NaN,NaN,None,NaN,1.697780e+14,37533178009,...,None,None,None,None,None,None,None,None,None,None
3370,NaN,None,barcelona,None,NaN,NaN,None,NaN,1.697782e+15,37504941105,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26904,NaN,None,barcelona,None,NaN,NaN,None,NaN,1.697943e+15,37522279243,...,None,None,None,None,None,None,None,None,None,None
27066,NaN,None,barcelona,None,NaN,NaN,None,NaN,1.697944e+14,37548656560,...,None,None,None,None,None,None,None,None,None,None
27114,NaN,None,barcelona,None,NaN,NaN,None,NaN,1.697944e+15,36762306409,...,None,None,None,None,None,None,None,None,None,None
27193,NaN,None,barcelona,None,NaN,NaN,None,NaN,1.697945e+15,37540936773,...,None,None,None,None,None,None,None,None,None,None
